# Sodankyla

#### Arctan loss, f_scale = .1

In [7]:
# from scripts.parallel_blocks import *
from scripts.parallel_phoreal import *
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Adjust the display width to show all columns
# pd.set_option('display.max_colwidth', None)  # Show full content of each column

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/sodankyla_full/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

all_ATL03

# coords = (26.634154, 67.361833)

# for i in range(N):
#     # print(all_ATL03[i], all_ATL08[i])
#     data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=5,height=5,\
#                          graph_detail=2,file_index=i,altitude=185,threshold=1,alt_thresh=80)
    # data = pvpg_parallel(dirpath, all_ATL03[i], all_ATL08[i], coords=coords, width=.01,height=.01,\
    #                      graph_detail=1,file_index=i,altitude=185,threshold=1,small_box=.01,alt_thresh=80)

['../data_store/data/sodankyla_full/processed_ATL03_20190112214359_02360205_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190124083414_04110203_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190327053800_13560203_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190413172354_02360305_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190425041400_04110303_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190614142731_11810305_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20190626011737_13560303_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20191023193336_04110503_006_02.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20200111042311_02360605_006_01.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20200122151319_04110603_006_01.h5',
 '../data_store/data/sodankyla_full/processed_ATL03_20200324121701_13560603_006_01.h5',
 '../data_store/data/sodankyla_f

# Delta Junction

In [4]:
from scripts.parallel_phoreal import *

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/delta_junction/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (-145.7514, 63.8811)

all_ATL03

# problematic: 0, 25, 48, 

# for i in range(N):
#     data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
#                          graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude=504)
#     # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
#     #                      graph_detail=2, loss='linear',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude=504)
# #     print(data)

['../data_store/data/delta_junction/processed_ATL03_20181115115748_07310105_006_02.h5',
 '../data_store/data/delta_junction/processed_ATL03_20190506151440_05860303_006_02.h5',
 '../data_store/data/delta_junction/processed_ATL03_20190516031713_07310305_006_02.h5',
 '../data_store/data/delta_junction/processed_ATL03_20190912213302_11730405_006_02.h5',
 '../data_store/data/delta_junction/processed_ATL03_20191113183651_07310505_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200105033758_01440603_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200312125239_11730605_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200503215348_05860703_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200513095624_07310705_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200704185731_01440803_006_01.h5',
 '../data_store/data/delta_junction/processed_ATL03_20200812053609_07310805_006_01.h5',
 '../data_store/data/delta_junct

# Marcell MN

In [2]:
from scripts.parallel_phoreal import *

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/marcell_MN/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (-93.4541, 47.5049)

all_ATL03

# for i in range(N):
#     data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
#                          graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude=413)
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='linear',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude=413)
#     print(data)

['../data_store/data/marcell_MN/processed_ATL03_20181104085355_05610106_006_02.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20190326134636_13460202_006_02.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20190924050607_13460402_006_02.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20191224004554_13460502_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20200201111246_05610606_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20200502065232_05610706_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20200622160526_13460702_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20200801023220_05610806_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20200921114515_13460802_006_02.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20201030221209_05610906_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20210129175203_05611006_006_01.h5',
 '../data_store/data/marcell_MN/processed_ATL03_20210221042856_09041002_006_

# Lacclair

In [5]:
from scripts.parallel_phoreal import *

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/lacclair/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (-71.6696, 46.9521)

all_ATL03

# for i in range(N):
#     data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
#                          graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude = 313)
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='linear',file_index=i, keep_flagged=1, opsys='bad',f_scale=.05, altitude = 313)
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.01, altitude = 313)
#     print(data)

['../data_store/data/lacclair/processed_ATL03_20190506102318_05830302_006_02.h5',
 '../data_store/data/lacclair/processed_ATL03_20190713192557_02400406_006_02.h5',
 '../data_store/data/lacclair/processed_ATL03_20190811180209_06820406_006_02.h5',
 '../data_store/data/lacclair/processed_ATL03_20190903043904_10250402_006_02.h5',
 '../data_store/data/lacclair/processed_ATL03_20191203001854_10250502_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20200202212238_05830602_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20200411062533_02400706_006_02.h5',
 '../data_store/data/lacclair/processed_ATL03_20200601153829_10250702_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20200711020519_02400806_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20200802124213_05830802_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20210131040156_05831002_006_01.h5',
 '../data_store/data/lacclair/processed_ATL03_20210206160107_06821006_006_01.h5',
 '../data_store/

# Torgnon

In [6]:
from scripts.parallel_phoreal import *

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/torgnon/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (7.5609, 45.8238)

all_ATL03

# for i in range(N):
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude = 2091)
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='linear',file_index=i, keep_flagged=1, opsys='bad',f_scale=.05, altitude = 2091)
#     print(data)

['../data_store/data/torgnon/processed_ATL03_20181130005516_09530106_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20190222083607_08540202_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20190228203518_09530206_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20190524041544_08540302_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20190530161453_09530306_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20190927103049_00080506_006_02.h5',
 '../data_store/data/torgnon/processed_ATL03_20200220151508_08540602_006_01.h5',
 '../data_store/data/torgnon/processed_ATL03_20200227031417_09530606_006_01.h5',
 '../data_store/data/torgnon/processed_ATL03_20200521105458_08540702_006_01.h5',
 '../data_store/data/torgnon/processed_ATL03_20200820063443_08540802_006_01.h5',
 '../data_store/data/torgnon/processed_ATL03_20200826183353_09530806_006_01.h5',
 '../data_store/data/torgnon/processed_ATL03_20201119021432_08540902_006_01.h5',
 '../data_store/data/torgnon

# Oregon Young Pine

In [8]:
from scripts.parallel_phoreal import *

# dirpath = 'R:/data/sodankyla_full/'
dirpath = '../data_store/data/oregon_yp/'

# data = []

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (-121.606, 44.3238)

all_ATL03

# for i in range(N):
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='arctan',file_index=i, keep_flagged=1, opsys='bad',f_scale=.1, altitude = 977)
    # data = pvpg_parallel(all_ATL03[i], all_ATL08[i], coords=coords, width=.1,height=.1,\
    #                      graph_detail=1, loss='linear',file_index=i, keep_flagged=1, opsys='bad',f_scale=.05, altitude = 977)
#     print(data)

['../data_store/data/oregon_yp/processed_ATL03_20190125065018_04250206_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20190620113430_12710302_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20190725220939_04250406_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20190823204548_08670406_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20190919071421_12710402_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20191024174938_04250506_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20200123132922_04250606_006_01.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20200218235753_08290602_006_01.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20200423090912_04250706_006_02.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20200617181343_12710702_006_01.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20200723044857_04250806_006_01.h5',
 '../data_store/data/oregon_yp/processed_ATL03_20201022002843_04250906_006_01.h5',
 '..